In [1]:
import subprocess
from pyspark.sql.types import IntegerType
from functools import reduce
from pyspark import SparkConf
from pyspark.sql import DataFrame,SparkSession
from pyspark.sql.functions import sum
#Intializing SparkSession and configuration for spark.config can also set here 
conf = SparkConf().setMaster("local").setAppName("test")
spark = SparkSession.builder.config(conf=conf).getOrCreate()



In [2]:
# Function that is created to to read the hdfs file using SparkSession
def hdfs_file_read(file):
    rdd = spark.read.csv(file, header = True)
    return rdd

In [3]:

#Function performs dataset computation and return dataframe for normalWeek and lockdownWeek  
def get_Dataframe(x):
    #This lines read the all names the file in hdfs folder and make it a single name with space seperated
    p = subprocess.Popen(x,shell=True,stdout=subprocess.PIPE,stderr=subprocess.STDOUT)
    res = list()
    #then the line send to this for loop to run line by line files this has 7 data for normal week and 7 data for lockdownweek
        #hdfs://localhost:9000/user/akash/trafficCount/aggre/data1.csv
        #hdfs://localhost:9000/user/akash/trafficCount/aggre/data2.csv
        #hdfs://localhost:9000/user/akash/trafficCount/aggre/Ldata1.csv
        #hdfs://localhost:9000/user/akash/trafficCount/aggre/Ldata1.csv
    for line in p.stdout.readlines():
        file = line.decode().strip()
        rt = hdfs_file_read(file)
        rt= rt.withColumn("VehicleCount", rt["VehicleCount"].cast(IntegerType()))
        dff1= rt.groupBy("class").sum("VehicleCount") #total count is groupby vehicle class
        dff=dff1.withColumnRenamed("sum(VehicleCount)", "TotalCount")
        res.append(dff)

    union_df = reduce(DataFrame.unionAll, res)
    union_df=union_df.groupBy("class").avg("TotalCount").sort('class')
    return union_df


In [4]:
w1="hdfs dfs -ls hdfs://localhost:9000/user/akash/trafficCount/aggre| awk '{print $8}' | awk 'NR!=1' "
w2="hdfs dfs -ls hdfs://localhost:9000/user/akash/trafficCount/lockdownweek| awk '{print $8}' | awk 'NR!=1' "
L=[w1,w2]
for i in range(0,2):
        j=str(i)
        globals()[f"week{j}"] =  get_Dataframe(L[i])       
#0----NULL
#1---M_BIKE
#2---car
#3---LGV
#4---BUS
#5---HGV_RIG
#6---HGV_ART
#7---CARAVAN

week0.show()#Normalweek 
week1.show()#lockdownweek

+-----+------------------+
|class|   avg(TotalCount)|
+-----+------------------+
|    0| 551.1428571428571|
|    1|           28564.0|
|    2| 6314142.714285715|
|    3| 814012.5714285715|
|    4|57620.142857142855|
|    5|175725.85714285713|
|    6|304124.71428571426|
|    7| 40229.57142857143|
+-----+------------------+

+-----+------------------+
|class|   avg(TotalCount)|
+-----+------------------+
|    0|204.57142857142858|
|    1| 9185.142857142857|
|    2|1463737.2857142857|
|    3|317359.28571428574|
|    4| 19941.85714285714|
|    5|           91562.0|
|    6|225943.14285714287|
|    7|14435.857142857143|
+-----+------------------+

